In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2592658019406236032, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 14912489397557601197
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 8633491846942947616
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 8802136339266664859
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 7429608409030987155
 physical_devic

In [3]:
import keras
import tensorflow as tf
import keras.backend as K
print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0
dim_ordering: tf


Using TensorFlow backend.


In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
import sys
import os
import numpy as np

from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization, GlobalAveragePooling2D


from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from keras.applications.xception import Xception, preprocess_input

from keras.regularizers import l2

from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
output_classes = 4
batch_size = 1
img_height, img_width = 224*5, 224*5
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 276
nb_validation_samples = 92
nb_test_samples = 32

In [ ]:
train_dir = 'data/processed-data/train/'
test_dir = 'data/processed-data/test/'

In [ ]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    validation_split=0.25)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    subset="training",
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    subset="validation",
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 276 images belonging to 4 classes.
Found 92 images belonging to 4 classes.
Found 32 images belonging to 4 classes.


In [ ]:
xception_weights = 'pretrained_models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

def xception_model():
    model = Sequential()
 
    model.add(Xception(weights = xception_weights , include_top=False,pooling = 'avg'))
    model.add(Dense(units=output_classes, activation=tf.nn.softmax))
    model.layers[0].trainable = True
    
    return model

In [ ]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001, 0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
         'decay':hp.choice('decay',[0.0, 0.004, 0.0001, 0.1, 0.3, 0.5]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [ ]:
def f_nn(params):   
    print ('Parameters testing: ', params)
    model = xception_model()
    
    adam_opt = Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=1e-08, decay=params['decay'])
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1, callbacks=get_callbacks(params))
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    
    th= 0.3
    acc = accuracy_score(truth, predict_class > th)
    print("Test Accuracy: {:.4f}".format(acc*100))
    print("*_*" * 50)

    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [ ]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_loss', patience=3, verbose=1)]
    return callbacks

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.3, 'beta_2': 0.99, 'decay': 0.0, 'lr': 0.0001}
Epoch 1/15
276/276 [==============================] - 126s 456ms/step - loss: 1.3612 - acc: 0.3225 - val_loss: 1.5489 - val_acc: 0.2935
Epoch 2/15
276/276 [==============================] - 122s 443ms/step - loss: 0.9477 - acc: 0.6486 - val_loss: 3.0390 - val_acc: 0.2826
Epoch 3/15
276/276 [==============================] - 122s 443ms/step - loss: 0.3858 - acc: 0.9167 - val_loss: 2.8338 - val_acc: 0.3804
Epoch 4/15
276/276 [==============================] - 122s 442ms/step - loss: 0.0857 - acc: 0.9891 - val_loss: 2.1268 - val_acc: 0.3587
Epoch 00004: early stopping
Validation Score:  2.210314267963404
Validation Accuracy:  0.4
32/32 [==============================] - 5s 144ms/step
No of errors = 22/32
Test Accuracy: 25.0000
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.